In [1]:
import pandas as pd

df_1 = pd.read_csv(r"C:\Users\anton\Dev\ABM\results20240820_072837kg_completeness_0_3kg_completeness_0_3_metrics_csv")
df_2 = pd.read_csv(r"C:\Users\anton\Dev\ABM\results20240820_072837kg_completeness_0_65kg_completeness_0_65_metrics_csv")
df_3 = pd.read_csv(r"C:\Users\anton\Dev\ABM\results20240820_072837kg_completeness_1_0kg_completeness_1_0_metrics_csv")

In [2]:
# def rename_columns(df, suffix):
#     df.columns = [f"{col}_{suffix}" for col in df.columns]
#     return df

# df_1 = rename_columns(df_1, '03')
# df_2 = rename_columns(df_2, '065')
# df_3 = rename_columns(df_3, '10')

In [3]:
df_1.head()

,Step,Performance,Game Manager Index,Best Route Energy,Curriculum Step,Target Route Energy
0,2048,8494.391111,4,inf,0,14
1,4096,7184.627976,7,inf,0,14
2,6144,8085.573254,11,inf,0,14
3,8192,13956.203313,14,inf,0,14
4,10240,82.605000,18,inf,0,14


In [4]:
# only keep the first two columns
df_1 = df_1.iloc[:, :2]
df_2 = df_2.iloc[:, :2]
df_3 = df_3.iloc[:, :2]

In [5]:
complete_df = pd.concat([df_1, df_2, df_3], axis=1)
complete_df.head()

,Step,Performance,Step,Performance,Step,Performance
0,2048.0,8494.391111,2048.0,5698.288889,2048,2827.030556
1,4096.0,7184.627976,4096.0,5453.028770,4096,11836.942897
2,6144.0,8085.573254,6144.0,12198.968254,6144,10417.224841
3,8192.0,13956.203313,8192.0,103.425000,8192,7266.666111
4,10240.0,82.605000,10240.0,9793.499643,10240,5552.270833


In [6]:
import pandas as pd
import plotly.graph_objects as go

def plot_performance_comparison(df_list, model_names=None):
    """
    Create an interactive Plotly plot comparing performance of models with steps as the x-axis,
    using Plotly's native axis handling capabilities.
    
    Parameters:
    df_list (list): List of DataFrames containing 'Step' and 'Performance' columns for each model
    model_names (list, optional): Names of the models for the legend. If None, default names will be used.
    
    Returns:
    plotly.graph_objects.Figure: The interactive plot
    """
    
    # Create the figure
    fig = go.Figure()
    
    # Generate default model names if not provided
    if model_names is None:
        model_names = [f'Model {i+1}' for i in range(len(df_list))]
    
    # Ensure we have enough model names
    if len(model_names) < len(df_list):
        model_names.extend([f'Model {i+1}' for i in range(len(model_names), len(df_list))])
    
    # Add traces for each model
    for df, name in zip(df_list, model_names):
        fig.add_trace(
            go.Scatter(x=df['Step'], y=df['Performance'], name=name, mode='lines+markers')
        )
    
    # Update layout
    fig.update_layout(
        title='Performance Comparison of Models',
        xaxis_title='Steps',
        yaxis_title='Performance',
        legend_title='Models',
        hovermode='x unified'
    )
    
    # Let Plotly handle the x-axis ticks
    fig.update_xaxes(
        type='linear',
        autorange=True,
        tickmode='auto',
        nticks=20  # Suggest the number of ticks, Plotly will optimize
    )
    
    return fig

# Example usage:
# df_list = [df_1, df_2, df_3]
# fig = plot_performance_comparison(df_list, model_names=['Model 0.3', 'Model 0.65', 'Model 1.0'])
# fig.show()

In [7]:
df_list = [df_1, df_2, df_3]
fig = plot_performance_comparison(df_list)
fig.show()

In [53]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def aggregate_data(df, window):
    """Aggregate data based on a moving window."""
    return df.rolling(window=int(window), on='Step', min_periods=1).mean().dropna()

def plot_performance_comparison(df_list, model_names=None, max_aggregation=50):
    """
    Create an interactive Plotly plot comparing performance of models with steps as the x-axis,
    including a slider for continuous data aggregation.
    
    Parameters:
    df_list (list): List of DataFrames containing 'Step' and 'Performance' columns for each model
    model_names (list, optional): Names of the models for the legend. If None, default names will be used.
    max_aggregation (int, optional): Maximum aggregation window size. Default is 50.
    
    Returns:
    plotly.graph_objects.Figure: The interactive plot
    """
    
    # Generate default model names if not provided
    if model_names is None:
        model_names = [f'Model {i+1}' for i in range(len(df_list))]
    
    # Ensure we have enough model names
    if len(model_names) < len(df_list):
        model_names.extend([f'Model {i+1}' for i in range(len(model_names), len(df_list))])
    
    # Create subplot with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Add traces for each model
    for df, name in zip(df_list, model_names):
        fig.add_trace(
            go.Scatter(x=df['Step'], y=df['Performance'], name=name, mode='lines+markers'),
            secondary_y=False
        )
    
    # Update layout
    fig.update_layout(
        title='Performance Comparison of Models',
        xaxis_title='Steps',
        yaxis_title='Performance',
        legend_title='Models',
        hovermode='x unified'
    )
    
    # Add range slider
    fig.update_layout(
        xaxis=dict(
            rangeslider=dict(visible=True),
            type="linear"
        )
    )
    
    # Add aggregation slider
    steps = []
    for i in range(max_aggregation):
        step = dict(
            method="update",
            args=[{"y": [aggregate_data(df, i+1)['Performance'] for df in df_list]}],
            label=str(i+1)
        )
        steps.append(step)

    sliders = [dict(
        active=0,
        currentvalue={"prefix": "Aggregation Window: "},
        pad={"t": 50},
        steps=steps
    )]
    
    fig.update_layout(
        sliders=sliders
    )
    
    return fig

# Example usage:
df_list = [df_1, df_2, df_3]
fig = plot_performance_comparison(df_list, model_names=['Model 0.3', 'Model 0.65', 'Model 1.0'], max_aggregation=50)
fig.show()

In [52]:
s = 'as.df9h.8a.ds.34.f34freg0*()_Y#NK<?=@#$%!#^&*()_+'
s = s.replace('.', '_')
s = ''.join(e for e in s if e.isalnum() or e == '_')
print(s)

as_df9h_8a_ds_34_f34freg0_YNK_
